In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [10]:
# Import gaming list
df = pd.read_csv (r'Games_owned.csv') #has a column named 'Link' with URLs for Metacritic for each game owned

In [5]:
#Scrape metacritic data
title = []
release_date = []
description = []
metascore = []
user_score = []
for game_url in df.Link:
    if isinstance(game_url, str): 
        if "https" in game_url: #If link is a URL, as some games/collections did not have a page
            user_agent = {'User-agent': 'Mozilla/5.0'}
            response  = requests.get(game_url, headers = user_agent)
            soup = BeautifulSoup(response.text, 'html.parser') #Use Soup to scrape site data
            title.append(soup.find('h1').text)
            release_date.append(soup.find('li', class_ = 'summary_detail release_data').find('span', class_ = 'data').text)
            if soup.find('span', class_ = 'blurb blurb_expanded') == None:
                description.append('No Summary')
            else:
                description.append(soup.find('span', class_ = 'blurb blurb_expanded').text)
            if soup.find('span', itemprop = 'ratingValue') == None:
                metascore.append('No Score') 
            else:
                metascore.append(soup.find('span', itemprop = 'ratingValue').text)
            if soup.find('div', class_=lambda class_: class_ and class_.startswith('metascore_w user')) == None:
                user_score.append('No Score') 
            else:
                user_score.append(soup.find('div', class_=lambda class_: class_ and class_.startswith('metascore_w user')).text)
            

In [7]:
#Build dataframe from lists
game_library = pd.DataFrame(list(zip(title,release_date,description,metascore,user_score)),columns = ['Title', 'Release Date', 
                                                                                                     'Description', 'MetaScore',
                                                                                                     'User Score'])

In [8]:
print(game_library)

                       Title        Release Date  \
0               Apex Legends        Feb  4, 2019   
1     Counter-Strike: Source        Oct  7, 2004   
2                  Destiny 2        Oct 24, 2017   
3                   Golf It!  TBA - Early Access   
4                Half-Life 2        Nov 16, 2004   
..                       ...                 ...   
330     Halo: Combat Evolved        Sep 30, 2003   
331                Fallout 3        Oct 28, 2008   
332  Tomb Raider: Underworld        Nov 18, 2008   
333                 BioShock        Aug 21, 2007   
334              Mass Effect        May 28, 2008   

                                           Description MetaScore User Score  
0    Conquer with character in Apex Legends, a free...        88        6.0  
1    Valve has taken a large selection of the maps ...        88        8.9  
2    In Destiny 2, the last safe city on Earth has ...        83        4.3  
3    Golf It! is a multiplayer Minigolf game with f...  No Scor

In [9]:
game_library.to_csv('game_library.csv')